<a href="https://colab.research.google.com/github/R-Allecks/SENAI_Python/blob/main/raspagem_de_dados_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#WEBSCRAPING

import requests
from bs4 import BeautifulSoup
import pandas as pd

#url para raspar os dados
url = 'https://books.toscrape.com/index.html'

#Fazer a requisição HTTP
response = requests.get(url)

In [ ]:
#Usar o beautifulSoup para parsear o conteudo (codifica HTML para Python)
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup.select('article.product_pod')

In [ ]:
#Totalizar itens:
len(soup.select('article.product_pod')) #Neste caso são 20 LIVROS no TOTAL

In [ ]:
#Selecionar itens:
soup.select('article.product_pod')[0] #Neste caso são 20 LIVROS no TOTAL

In [ ]:
#Totalizar itens:
soup.select('article.product_pod')[0].h3.a['title'] #Neste caso são 20 LIVROS no TOTAL

In [ ]:
#preço:
soup.select('article.product_pod')[0].select_one('p.price_color').text #Neste caso são 20 LIVROS no TOTAL

In [ ]:
#preço:
#Original: soup.select('article.product_pod')[0].select_one('p.instock availability').text
#The original selector is likely incorrect. There's probably a space in the class name
#and "instock" and "availability" are two separate classes.
#Corrected:
soup.select('article.product_pod')[0].select_one('p.instock.availability').text #Neste caso são 20 LIVROS no TOTAL

In [ ]:
star = soup.select('article.product_pod')[0].select_one('p.star-rating')
star.get('class')[1]

In [ ]:
#traduzir
def traduzir_numero(numero_em_ingles):
    traducoes = {
        'one': 'um',
        'two': 'dois',
        'three': 'três',
        'four': 'quatro',
        'five': 'cinco'
    }
    #Retorna a tradução ou uma mensagem caso o numero não esteja no dicionario
    return traducoes.get(numero_em_ingles.lower(), 'Número não encontrado')

In [ ]:
traduzir_numero(star.get('class')[1])

In [ ]:
# DESAFIO

# URL  para raspar os dados
url = 'https://books.toscrape.com/index.html'

# fazer a requisição HTTP
response = requests.get(url)

In [ ]:
ls_titulo = []
ls_preco = []
ls_estoque = []
ls_nota = []


for book in soup.select('article.product_pod'):
    ls_titulo.append(book.h3.a['title'])
    ls_preco.append(book.select_one('p.price_color').text)
    ls_estoque.append(book.select_one('p.instock.availability').text.strip())

    # pegar estrela
    star = book.select_one('p.star-rating')
    ls_nota.append(traduzir_numero(star.get('class')[1]))

pagina = pd.DataFrame({'titulo': ls_titulo,
                       'preco': ls_preco,
                       'estoque': ls_estoque,
                       'nota': ls_nota
})



In [ ]:
pagina.to_excel('book.xlsx')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# URL base para navegação
url_base = 'https://books.toscrape.com/catalogue/'

# Listas para armazenar os dados
ls_titulo = []
ls_preco = []
ls_estoque = []
ls_nota = []

# Loop para percorrer todas as páginas
pagina_atual = 1
while True:
    # URL da página atual
    url = f'{url_base}page-{pagina_atual}.html'
    response = requests.get(url)

    # Verifica se a página existe
    if response.status_code != 200:
        print(f"Fim das páginas na página {pagina_atual}.")
        break

    # Faz a raspagem dos dados
    soup = BeautifulSoup(response.text, 'html.parser')
    for book in soup.select('article.product_pod'):
        ls_titulo.append(book.h3.a['title'])
        ls_preco.append(book.select_one('p.price_color').text)
        ls_estoque.append(book.select_one('p.instock.availability').text.strip())

        # Pegar a nota (estrelas)
        star = book.select_one('p.star-rating')
        ls_nota.append(traduzir_numero(star.get('class')[1].lower()))

    print(f"Página {pagina_atual} raspada.")
    pagina_atual += 1

# Criar o DataFrame com os dados coletados
pagina = pd.DataFrame({
    'Titulo': ls_titulo,
    'Preço': ls_preco,
    'Estoque': ls_estoque,
    'Nota': ls_nota
})

# Exibir os primeiros registros
print(pagina.head())

In [ ]:
pagina.shape

In [ ]:
pagina.to_excel('book.xlsx')

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('/content/book.xlsx')

In [ ]:
#Tratamento e limpeza de dados
#Tratamento e limpeza de dados
#limpeza de dados
#1. converter a coluna "PRECO" de string para float
df['Preço'] = df['Preço'].str.replace('£', '', regex=True).replace('[Â]', ' ', regex=True).astype(float)

#2 Converter a coluna "NOT" DE TEXTO PARA VALORES NUMERICOS
nota_mapa = {'um': 1, 'dois': 2, 'três': 3, 'quatro': 4, 'cinco': 5}
df['Nota'] = df['Nota'].map(nota_mapa)


In [ ]:
df

In [ ]:
#Analise estatiscicas com PANDAS
#Media, mediana, e desvio padrão
media_preco = df['Preço'].mean()
mediana_preco = df['Preço'].median()
desvio_preco = df['Preço'].std()

#Preço medio por nota
media_preco_por_nota = df.groupby('Nota')['Preço'].mean()

#Resultados Estatisticos
print('Média dos preços: £{:.2f}' .format(media_preco))
print('Mediana dos preços: £{:.2f}' .format(mediana_preco))
print('Desvio padrão dos preços: £{:.2f}' .format(desvio_preco))
print('\nPreço médio por nota:')
print(media_preco_por_nota)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponha que seu DataFrame seja chamado 'df' e a coluna de preços seja 'Preço'

# 1. Criar o histograma
plt.figure(figsize=(8, 5))  # Define o tamanho da figura
sns.boxplot(x='Nota', y='Preço', data=df, palette='viridis')  # Define a paleta de cores
plt.title('Distribuição de Preços por Nota')
plt.xlabel('Nota')
plt.ylabel('Preço')
plt.show()